# Interactive Occlusion Heatmap slider for all models

For a given patient id the occlusion heatmap is calculated for all models and displayed in a slider.
The size of the occlusion can be adjusted manually.

### Imports

In [ ]:
%matplotlib inline

import os
import numpy as np

import tensorflow as tf

print("TF  Version",tf.__version__)

In [ ]:
# check and set path before loading modules
print(os.getcwd())
INPUT_DIR = "/tf/notebooks/schnemau/xAI_stroke_3d/"
OUTPUT_DIR = "/tf/notebooks/bule/explainable_AI/"
if os.getcwd() != OUTPUT_DIR:
    os.chdir(OUTPUT_DIR)

In [ ]:
import functions_model_definition as md
import functions_read_data as rdat
import functions_occlusion as oc
import functions_gradcam as gc
import functions_plot_heatmap as phm
import functions_slider as sl

### Load Data and Set Parameters

In [ ]:
## ToDo: 
## - hm_type should always be "gc" in this notebook
## - pred_hm_only, norm_hm and hm_mode should only be adjusted in last chunk
## - pic_save_name is not needed in this notebook: implement dictionary for paths

# Define Version
version = "CIBLSX" # one of:
# 10Fold_sigmoid_V0, 10Fold_sigmoid_V1, 10Fold_sigmoid_V2, 10Fold_sigmoid_V2f, 10Fold_sigmoid_V3
# 10Fold_softmax_V0, 10Fold_softmax_V1, andrea
# 10Fold_CIB, 10Fold_CIBLSX

# Define Model Version
model_version = 3

# define weighting
hm_mode = "wgt" 

# define heatmap type
hm_type = "oc"
norm_hm = False
pred_hm_only = True

# Select naming convention (for CIBLSX model_version >= 3 and CIB model_version >= 2 should be False, else True)
comp_mode = False # if True: use old naming convention

# define paths
DATA_DIR, WEIGHT_DIR, DATA_OUTPUT_DIR, PIC_OUTPUT_DIR, pic_save_name = rdat.dir_setup(
    INPUT_DIR, OUTPUT_DIR, version, model_version, 
    weight_mode = hm_mode, hm_type = hm_type, pred_hm = pred_hm_only, hm_norm = norm_hm,
    compatibility_mode=comp_mode)

In [ ]:
## load images and ids
(X_in, pat_ids, id_tab, all_results_tab, pat_orig_tab, pat_norm_tab, num_models) = rdat.version_setup(
    DATA_DIR = DATA_DIR, version = version, model_version = model_version,
    compatibility_mode=comp_mode)

## Model

In [ ]:
# define model
(input_dim_img, output_dim, LOSS, layer_connection, last_activation) = md.model_setup(version)

model_3d = md.model_init(
    version = version, 
    output_dim = output_dim,
    LOSS = LOSS,
    layer_connection = layer_connection,
    last_activation = last_activation,
    C = 2,
    learning_rate = 5*1e-5,
    batch_size = 6,
    input_dim = input_dim_img,
    input_dim_tab = pat_norm_tab.drop(columns=["p_id"]).shape[1] if "LSX" in version else None,
)

In [ ]:
# Define Model Name
generate_model_name = md.set_generate_model_name(
    model_version = model_version, 
    layer_connection = layer_connection, 
    last_activation = last_activation, 
    path = WEIGHT_DIR,
    compatability_mode=comp_mode)  

### Check Occlusion Size

In [ ]:
### occlusion
# occ_size = (16, 16, 12)
# occ_stride = 8
# occ_size = (14, 14, 10)
# occ_stride = (6)
occ_size = (18, 18, 4)
occ_stride = (10, 10, 3)
print("number of occlusions: ", int(np.prod(((np.array(X_in.shape[1:4]) - occ_size) / occ_stride) + 1)))
print("number of occlusions per axis: ", ((np.array(X_in.shape[1:4]) - occ_size) / occ_stride) + 1)
print((np.asarray(X_in.shape[1:4]) - occ_size) % occ_stride) # all must be zero
print(all(np.array(occ_size) > occ_stride)) # must be true
print(np.array(X_in.shape[1:4]) / occ_size) # if all same, then same ratio of occ_size to image size

# Occlusion Slider

In [ ]:
## old occlusion
# occ_size = (16, 16, 12)
# occ_stride = [8]
# occ_size = (14, 14, 10)
# occ_stride = [6]

## new occlusion
# occ_size = (20, 20, 16)
# occ_stride = [6]
occ_size = (18, 18, 4)
occ_stride = (10, 10, 3)

sl.occlusion_interactive_plot(
    6, # patient id
    occ_size = occ_size, occ_stride = occ_stride,
    cnn=model_3d, all_results=all_results_tab, pat=pat_ids, X_in=X_in,
    generate_model_name=generate_model_name, num_models=num_models,
    pat_dat=pat_orig_tab, pat_norm_table=pat_norm_tab,
    model_mode="weighted",
    normalize_hm=False,
    pred_hm_only=True) # if True, only the heatmap for the predicted class is shown, otherwise also negative heatmaps are shown

In [ ]:
import matplotlib.pyplot as plt
im = X_in[np.where(all_results_tab.p_id == 6)[0]][0, :, :, 4]
plt.imshow(im, cmap="gray")

In [ ]:
np.quantile(X_in, q = [0.25, 0.5, 0.75])

In [ ]:
im_occ = im.copy()
im_occ[70:70+occ_size[0], 60:60+occ_size[1]] = 0
plt.imshow(im_occ, cmap="gray")